In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import seaborn as sns
%matplotlib inline
import warnings
from sklearn import preprocessing

warnings.filterwarnings('ignore')

In [3]:
ds = pl.read_csv('./course_project_train.csv')
y = ds['Credit Default']
X = ds.drop(['Credit Default'])

In [4]:
X.head()

Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score
str,f64,str,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64
"""Own Home""",482087.0,null,0.0,11.0,26.3,685960.0,1.0,null,1.0,"""debt consolida…","""Short Term""",9.9999999e7,47386.0,7914.0,749.0
"""Own Home""",1.025487e6,"""10+ years""",0.0,15.0,15.3,1.18173e6,0.0,null,0.0,"""debt consolida…","""Long Term""",264968.0,394972.0,18373.0,737.0
"""Home Mortgage""",751412.0,"""8 years""",0.0,11.0,35.0,1.182434e6,0.0,null,0.0,"""debt consolida…","""Short Term""",9.9999999e7,308389.0,13651.0,742.0
"""Own Home""",805068.0,"""6 years""",0.0,8.0,22.5,147400.0,1.0,null,1.0,"""debt consolida…","""Short Term""",121396.0,95855.0,11338.0,694.0
"""Rent""",776264.0,"""8 years""",0.0,13.0,13.6,385836.0,1.0,null,0.0,"""debt consolida…","""Short Term""",125840.0,93309.0,7180.0,719.0


In [5]:
# Проверка типов полей
X.describe()

describe,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score
str,str,f64,str,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64
"""count""","""7500""",7500.0,"""7500""",7500.0,7500.0,7500.0,7500.0,7500.0,7500.0,7500.0,"""7500""","""7500""",7500.0,7500.0,7500.0,7500.0
"""null_count""","""0""",1557.0,"""371""",0.0,0.0,0.0,0.0,0.0,4081.0,14.0,"""0""","""0""",0.0,0.0,0.0,1557.0
"""mean""",null,1.3664e6,null,0.030133,11.130933,18.317467,945153.727467,0.17,34.6926,0.117152,null,null,1.1873e7,289833.2352,18314.454133,1151.087498
"""std""",null,845339.199634,null,0.271604,4.908924,7.041946,1.6026e7,0.498598,21.688806,0.347192,null,null,3.1926e7,317871.38238,11926.764673,1604.451418
"""min""","""Have Mortgage""",164597.0,"""1 year""",0.0,2.0,4.0,0.0,0.0,0.0,0.0,"""business loan""","""Long Term""",11242.0,0.0,0.0,585.0
"""max""","""Rent""",1.0149344e7,"""< 1 year""",7.0,43.0,57.7,1.3047e9,7.0,118.0,4.0,"""wedding""","""Short Term""",9.9999999e7,6.506797e6,136679.0,7510.0
"""median""",null,1.168386e6,null,0.0,10.0,17.0,478159.0,0.0,32.0,0.0,null,null,309573.0,209323.0,16076.5,731.0
"""25%""",null,844284.0,null,0.0,8.0,13.5,279246.0,0.0,16.0,0.0,null,null,180180.0,114266.0,10068.0,711.0
"""75%""",null,1.640232e6,null,0.0,14.0,21.8,793716.0,0.0,50.0,0.0,null,null,520410.0,360449.0,23836.0,743.0


In [6]:
X = X.with_columns(pl.col("Tax Liens").cast(pl.Int32))
X = X.with_columns(pl.col("Months since last delinquent").cast(pl.Int32))

In [7]:
X.schema

{'Home Ownership': Utf8,
 'Annual Income': Float64,
 'Years in current job': Utf8,
 'Tax Liens': Int32,
 'Number of Open Accounts': Float64,
 'Years of Credit History': Float64,
 'Maximum Open Credit': Float64,
 'Number of Credit Problems': Float64,
 'Months since last delinquent': Int32,
 'Bankruptcies': Float64,
 'Purpose': Utf8,
 'Term': Utf8,
 'Current Loan Amount': Float64,
 'Current Credit Balance': Float64,
 'Monthly Debt': Float64,
 'Credit Score': Float64}

In [8]:
float_columns = X.select([pl.col(pl.Float64)]).columns
float_columns

['Annual Income',
 'Number of Open Accounts',
 'Years of Credit History',
 'Maximum Open Credit',
 'Number of Credit Problems',
 'Bankruptcies',
 'Current Loan Amount',
 'Current Credit Balance',
 'Monthly Debt',
 'Credit Score']

Проверим на вылеты

In [9]:
# sns.set()
# # plt.figure(figsize = (16,9))
# for float_column in float_columns:
#     sns.boxplot( data = X[float_column]).set(xlabel=float_column)
#     plt.show()

In [32]:
X.null_count()

Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,literal
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7244


In [31]:
X = X.with_columns(
    pl.col('Annual Income').fill_null(pl.mean('Annual Income')),
    pl.col('Years in current job').fill_null(0),
    pl.col('Months since last delinquent').fill_null(0),
    pl.col('Bankruptcies').fill_null(0),
    pl.col('Credit Score').fill_null(0)
)

In [23]:
for float_column in float_columns:
    q1 = np.percentile(X[float_column], 25, interpolation='midpoint')
    q3 = np.percentile(X[float_column], 75, interpolation='midpoint')
    iqr = q3 - q1
    upper = q3 + 1.5 * q1
    X = X.with_columns(
        pl.when(pl.any(pl.col(float_column) > upper))
        .then(upper)
        .otherwise(pl.col(float_column)).alias(float_column)
    )


In [38]:
scaler = preprocessing.StandardScaler()

X_norm_float = pl.DataFrame(scaler.fit_transform(X[float_columns]))
X_norm_float.columns = float_columns
X_norm_float.head()

Annual Income,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Bankruptcies,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
-1.492958,-0.018313,1.155682,0.363231,0.0,2.545732,1.845862,-1.222161,-0.981883,-0.108568
-0.530004,0.825615,-0.431231,1.787092,0.0,-0.337099,-0.433364,0.919203,0.071706,-0.116555
-1.01569,-0.018313,2.410786,1.789114,0.0,-0.337099,1.845862,0.385793,-0.403965,-0.113227
-0.920606,-0.651259,0.607476,-1.183523,0.0,2.545732,-1.056277,-0.923559,-0.636966,-0.145174
-0.971649,0.403651,-0.676481,-0.498731,0.0,-0.337099,-1.036996,-0.939244,-1.055823,-0.128535


In [41]:
pl.concat([X, X_norm_float])

ShapeError: unable to append to a dataframe of width 17 with a dataframe of width 10